In [ ]:
import React, { useEffect, useState } from 'react';

export default function TransactionMonitoring() {
  const [users, setUsers] = useState([]);
  const [selectedUserId, setSelectedUserId] = useState('');
  const [transactions, setTransactions] = useState([]);
  const [showTransactions, setShowTransactions] = useState(false);
  const [loading, setLoading] = useState(false);
  const [processStatuses, setProcessStatuses] = useState({});
  const [selectedTransaction, setSelectedTransaction] = useState(null);

  useEffect(() => {
    fetch('http://localhost:3000/users')
      .then((res) => res.json())
      .then((data) => setUsers(data))
      .catch((err) => console.error('Error fetching users:', err));
  }, []);

  const handleViewTransactions = async () => {
    if (!selectedUserId) return;
    setLoading(true);
    try {
      const res = await fetch(`http://localhost:3000/transactions/user/${selectedUserId}`);
      const data = await res.json();
      const suspicious = filterSuspiciousTransactions(data);
      setTransactions(suspicious);
      setShowTransactions(true);

      const initialStatuses = {};
      suspicious.forEach(tx => {
        initialStatuses[tx.transaction_id] = 'Pending';
      });
      setProcessStatuses(initialStatuses);
    } catch (err) {
      console.error('Error fetching transactions:', err);
    }
    setLoading(false);
  };

  const filterSuspiciousTransactions = (data) => {
    const dateCount = {};
    const keywords = ['Politics Funds', 'NGO'];

    data.forEach((tx) => {
      const dateOnly = tx.transaction_date.split('T')[0];
      dateCount[dateOnly] = (dateCount[dateOnly] || 0) + 1;
    });

    return data
      .map((tx) => {
        const dateOnly = tx.transaction_date.split('T')[0];
        const isHighAmount = parseFloat(tx.transaction_amount) > 200;
        const isFrequent = dateCount[dateOnly] > 3;
        const hasKeyword = keywords.some((kw) =>
          tx.transaction_description?.toLowerCase().includes(kw.toLowerCase())
        );

        const suspicionScore = [isHighAmount, isFrequent, hasKeyword].filter(Boolean).length;

        let suspicionLevel = '';
        if (suspicionScore === 3) suspicionLevel = 'Most Likely';
        else if (suspicionScore === 2) suspicionLevel = 'More Likely';
        else if (suspicionScore === 1) suspicionLevel = 'Likely';

        return suspicionScore > 0
          ? { ...tx, suspicion_level: suspicionLevel }
          : null;
      })
      .filter(Boolean);
  };

  const handleProcessStatusChange = (transactionId, newStatus) => {
    setProcessStatuses(prev => ({
      ...prev,
      [transactionId]: newStatus
    }));

    // Save to backend if marked as Completed
    if (newStatus === 'Completed') {
      const completedTx = transactions.find(tx => tx.transaction_id === transactionId);

      fetch('http://localhost:3000/completed-transactions', {
        method: 'POST',
        headers: { 'Content-Type': 'application/json' },
        body: JSON.stringify(completedTx)
      })
        .then(res => {
          if (!res.ok) throw new Error('Failed to save completed transaction');
          console.log('Completed transaction saved to backend.');
        })
        .catch(err => console.error('Error saving completed transaction:', err));
    }
  };

  const handleViewDetails = (transaction) => {
    setSelectedTransaction(transaction);
  };

  const getBadgeStyle = (level) => {
    switch (level) {
      case 'Most Likely':
        return 'bg-red-500 text-white';
      case 'More Likely':
        return 'bg-yellow-500 text-white';
      case 'Likely':
        return 'bg-blue-500 text-white';
      default:
        return 'bg-gray-400 text-white';
    }
  };

  const activeTransactions = transactions.filter(
    (tx) => processStatuses[tx.transaction_id] !== 'Completed'
  );

  const completedTransactions = transactions.filter(
    (tx) => processStatuses[tx.transaction_id] === 'Completed'
  );

  return (
    <div className="max-w-7xl mx-auto p-6 bg-white rounded-xl shadow-lg mt-10">
      <h2 className="text-3xl font-bold mb-6 text-center">Transaction Monitoring</h2>

      <div className="flex gap-4 mb-6">
        <select
          value={selectedUserId}
          onChange={(e) => setSelectedUserId(e.target.value)}
          className="w-3/4 px-3 py-2 border rounded"
        >
          <option value="">Select Account</option>
          {users.map((user) => (
            <option key={user.user_id} value={user.user_id}>
              {user.name}
            </option>
          ))}
        </select>

        <button
          onClick={handleViewTransactions}
          disabled={!selectedUserId}
          className={`w-1/4 px-4 py-2 rounded transition ${
            selectedUserId ? 'bg-blue-600 text-white hover:bg-blue-700' : 'bg-gray-400 text-white cursor-not-allowed'
          }`}
        >
          View Transactions
        </button>
      </div>

      {loading && (
        <p className="text-center text-blue-500 mb-4 animate-pulse">
          Loading transactions...
        </p>
      )}

      {showTransactions && (
        <div className="overflow-x-auto mt-6">
          <h3 className="text-2xl font-semibold mb-4 text-red-600">Suspicious Transactions</h3>

          <table className="w-full table-auto border border-gray-300 mb-6">
            <thead className="bg-gray-100">
              <tr>
                <th className="px-4 py-2 border">Transaction ID</th>
                <th className="px-4 py-2 border">Amount</th>
                <th className="px-4 py-2 border">Date</th>
                <th className="px-4 py-2 border">Status</th>
                <th className="px-4 py-2 border">Description</th>
                <th className="px-4 py-2 border">Suspicion Level</th>
                <th className="px-4 py-2 border">Process Status</th>
                <th className="px-4 py-2 border">Action</th>
              </tr>
            </thead>
            <tbody>
              {activeTransactions.map((tx) => (
                <tr key={tx.transaction_id} className="hover:bg-gray-100">
                  <td className="px-4 py-2 border text-center">{tx.transaction_id}</td>
                  <td className="px-4 py-2 border text-center">${tx.transaction_amount}</td>
                  <td className="px-4 py-2 border text-center">{tx.transaction_date.split('T')[0]}</td>
                  <td className="px-4 py-2 border text-center">{tx.transaction_status}</td>
                  <td className="px-4 py-2 border text-center truncate max-w-xs">{tx.transaction_description}</td>
                  <td className="px-4 py-2 border text-center">
                    <span className={`px-3 py-1 rounded-full text-sm font-semibold ${getBadgeStyle(tx.suspicion_level)}`}>
                      {tx.suspicion_level}
                    </span>
                  </td>
                  <td className="px-4 py-2 border text-center">
                    <select
                      value={processStatuses[tx.transaction_id] || 'Pending'}
                      onChange={(e) => handleProcessStatusChange(tx.transaction_id, e.target.value)}
                      className="px-2 py-1 border rounded"
                    >
                      <option value="Pending">Pending</option>
                      <option value="In Progress">In Progress</option>
                      <option value="Completed">Completed</option>
                    </select>
                  </td>
                  <td className="px-4 py-2 border text-center">
                    <button
                      onClick={() => handleViewDetails(tx)}
                      className="bg-blue-500 hover:bg-blue-700 text-white text-sm px-3 py-1 rounded"
                    >
                      View
                    </button>
                  </td>
                </tr>
              ))}
            </tbody>
          </table>
        </div>
      )}

      {/* Transaction Detail Modal */}
      {selectedTransaction && (
        <div
          className="fixed inset-0 bg-black bg-opacity-40 flex items-center justify-center z-50"
          onClick={() => setSelectedTransaction(null)}
        >
          <div
            className="bg-white p-6 rounded-lg shadow-xl w-full max-w-md"
            onClick={(e) => e.stopPropagation()}
          >
            <h3 className="text-xl font-bold mb-4">Transaction Details</h3>
            <p><strong>ID:</strong> {selectedTransaction.transaction_id}</p>
            <p><strong>Amount:</strong> ${selectedTransaction.transaction_amount}</p>
            <p><strong>Date:</strong> {selectedTransaction.transaction_date.split('T')[0]}</p>
            <p><strong>Status:</strong> {selectedTransaction.transaction_status}</p>
            <p><strong>Description:</strong> {selectedTransaction.transaction_description}</p>
            <p><strong>Suspicion Level:</strong> {selectedTransaction.suspicion_level}</p>
            <p><strong>Process Status:</strong> {processStatuses[selectedTransaction.transaction_id]}</p>
            <button
              onClick={() => setSelectedTransaction(null)}
              className="mt-6 bg-red-500 hover:bg-red-700 text-white px-4 py-2 rounded w-full"
            >
              Close
            </button>
          </div>
        </div>
      )}
    </div>
  );
}


SyntaxError: invalid syntax (<ipython-input-2-5420560e47cf>, line 1)

In [ ]:
## user list

import React, { useEffect, useState } from 'react';

export default function RegisteredUsers() {
  const [users, setUsers] = useState([]);
  const [search, setSearch] = useState('');
  const [selectedUser, setSelectedUser] = useState(null);

  useEffect(() => {
    fetch('http://localhost:3000/users') // Replace with your actual endpoint
      .then((res) => res.json())
      .then((data) => setUsers(data))
      .catch((err) => console.error('Error fetching users:', err));
  }, []);

  const filteredUsers = users.filter((user) =>
    Object.values(user).some((val) =>
      val?.toString().toLowerCase().includes(search.toLowerCase())
    )
  );

  const getBadgeClass = (type) => {
    switch (type?.toLowerCase()) {
      case 'passport':
        return 'bg-green-100 text-green-800';
      case 'driver license':
        return 'bg-blue-100 text-blue-800';
      case 'national id':
        return 'bg-yellow-100 text-yellow-800';
      default:
        return 'bg-gray-200 text-gray-700';
    }
  };

  return (
    <div className="max-w-6xl mx-auto p-6 bg-white shadow rounded-lg mt-10">
      <h2 className="text-3xl font-bold mb-4 text-center">All Registered Users</h2>

      <input
        type="text"
        placeholder="Search by any field"
        value={search}
        onChange={(e) => setSearch(e.target.value)}
        className="w-full px-4 py-2 border mb-6 rounded"
      />

      <div className="overflow-x-auto">
        <table className="w-full table-auto border border-gray-300">
          <thead className="bg-blue-50">
            <tr>
              <th className="px-4 py-2 border">Name</th>
              <th className="px-4 py-2 border">Email</th>
              <th className="px-4 py-2 border">Phone</th>
              <th className="px-4 py-2 border">Address</th>
              <th className="px-4 py-2 border">ID Type</th>
              <th className="px-4 py-2 border">ID Number</th>
              <th className="px-4 py-2 border">Action</th>
            </tr>
          </thead>
          <tbody>
            {filteredUsers.length > 0 ? (
              filteredUsers.map((user, index) => (
                <tr key={index} className="hover:bg-gray-100">
                  <td className="px-4 py-2 border">{user.name}</td>
                  <td className="px-4 py-2 border">{user.email}</td>
                  <td className="px-4 py-2 border">{user.phone || 'N/A'}</td>
                  <td className="px-4 py-2 border">{user.address || 'N/A'}</td>
                  <td className="px-4 py-2 border text-center">
                    <span className={`px-2 py-1 text-xs rounded-full ${getBadgeClass(user.id_type)}`}>
                      {user.id_type || 'N/A'}
                    </span>
                  </td>
                  <td className="px-4 py-2 border">{user.id_number || 'N/A'}</td>
                  <td className="px-4 py-2 border text-center">
                    <button
                      onClick={() => setSelectedUser(user)}
                      className="text-blue-600 hover:underline text-sm"
                    >
                      View
                    </button>
                  </td>
                </tr>
              ))
            ) : (
              <tr>
                <td colSpan="7" className="text-center py-4 text-gray-500">
                  No users found.
                </td>
              </tr>
            )}
          </tbody>
        </table>
      </div>

      {/* User Detail Modal */}
      {selectedUser && (
        <div
          className="fixed inset-0 bg-black bg-opacity-50 flex items-center justify-center z-50"
          onClick={() => setSelectedUser(null)}
        >
          <div
            className="bg-white p-6 rounded shadow-lg w-full max-w-md"
            onClick={(e) => e.stopPropagation()}
          >
            <h3 className="text-xl font-bold mb-4">User Details</h3>
            <p><strong>Name:</strong> {selectedUser.name}</p>
            <p><strong>Email:</strong> {selectedUser.email}</p>
            <p><strong>Phone:</strong> {selectedUser.phone || 'N/A'}</p>
            <p><strong>Address:</strong> {selectedUser.address || 'N/A'}</p>
            <p><strong>ID Type:</strong> {selectedUser.id_type || 'N/A'}</p>
            <p><strong>ID Number:</strong> {selectedUser.id_number || 'N/A'}</p>

            <button
              onClick={() => setSelectedUser(null)}
              className="mt-4 w-full bg-blue-600 text-white px-4 py-2 rounded hover:bg-blue-700"
            >
              Close
            </button>
          </div>
        </div>
      )}
    </div>
  );
}


SyntaxError: invalid syntax (<ipython-input-1-70aabf53eaa2>, line 3)

In [ ]:
### user form

import React, { useState } from 'react';

export default function UserForm() {
  const initialFormData = {
    name: '',
    dob: '',
    gender: '',
    nationality: '',
    email: '',
    phone: '',
    street: '',
    city: '',
    state: '',
    postal_code: '',
    country: '',
    id_type: '',
    id_number: '',
    expiry_date: ''
  };

  const [formData, setFormData] = useState(initialFormData);
  const [loading, setLoading] = useState(false);

  const handleChange = (e) => {
    setFormData({
      ...formData,
      [e.target.name]: e.target.value
    });
  };

  const handleSubmit = async (e) => {
    e.preventDefault();
    setLoading(true);

    const payload = {
      name: formData.name,
      dob: formData.dob,
      gender: formData.gender,
      nationality: formData.nationality,
      email: formData.email,
      phone: formData.phone,
      address: {
        street: formData.street,
        city: formData.city,
        state: formData.state,
        postal_code: formData.postal_code,
        country: formData.country
      },
      identification: {
        id_type: formData.id_type,
        id_number: formData.id_number,
        expiry_date: formData.expiry_date
      }
    };

    try {
      const res = await fetch('http://localhost:3000/users', {
        method: 'POST',
        headers: { 'Content-Type': 'application/json' },
        body: JSON.stringify(payload)
      });

      const data = await res.json();
      setLoading(false);

      if (!res.ok) {
        alert(`Error: ${data.error || 'Something went wrong'}`);
        return;
      }

      alert('User saved successfully!');
      setFormData(initialFormData);
    } catch (error) {
      setLoading(false);
      alert(`Request failed: ${error.message}`);
    }
  };

  return (
    <div className="max-w-4xl mx-auto p-8 bg-gradient-to-br from-white via-gray-50 to-gray-100 rounded-2xl shadow-2xl mt-10">
      <h2 className="text-3xl font-extrabold text-center bg-gradient-to-r from-blue-600 to-indigo-600 text-transparent bg-clip-text mb-8">
        User Registration
      </h2>

      <form onSubmit={handleSubmit} className="grid grid-cols-1 md:grid-cols-2 gap-6">
        {/* Personal Info */}
        <div>
          <label className="block mb-1 font-medium text-gray-700">Full Name</label>
          <input
            type="text"
            name="name"
            value={formData.name}
            onChange={handleChange}
            required
            className="w-full px-4 py-2 border rounded-lg shadow-sm focus:outline-none focus:ring-2 focus:ring-blue-500"
          />
        </div>
        <div>
          <label className="block mb-1 font-medium text-gray-700">Date of Birth</label>
          <input
            type="date"
            name="dob"
            value={formData.dob}
            onChange={handleChange}
            required
            className="w-full px-4 py-2 border rounded-lg shadow-sm focus:outline-none focus:ring-2 focus:ring-blue-500"
          />
        </div>
        <div>
          <label className="block mb-1 font-medium text-gray-700">Gender</label>
          <select
            name="gender"
            value={formData.gender}
            onChange={handleChange}
            required
            className="w-full px-4 py-2 border rounded-lg shadow-sm focus:outline-none focus:ring-2 focus:ring-blue-500"
          >
            <option value="">Select</option>
            <option>Male</option>
            <option>Female</option>
            <option>Other</option>
          </select>
        </div>
        <div>
          <label className="block mb-1 font-medium text-gray-700">Nationality</label>
          <input
            type="text"
            name="nationality"
            value={formData.nationality}
            onChange={handleChange}
            required
            className="w-full px-4 py-2 border rounded-lg shadow-sm focus:outline-none focus:ring-2 focus:ring-blue-500"
          />
        </div>
        <div>
          <label className="block mb-1 font-medium text-gray-700">Email</label>
          <input
            type="email"
            name="email"
            value={formData.email}
            onChange={handleChange}
            required
            className="w-full px-4 py-2 border rounded-lg shadow-sm focus:outline-none focus:ring-2 focus:ring-blue-500"
          />
        </div>
        <div>
          <label className="block mb-1 font-medium text-gray-700">Phone</label>
          <input
            type="tel"
            name="phone"
            value={formData.phone}
            onChange={handleChange}
            required
            className="w-full px-4 py-2 border rounded-lg shadow-sm focus:outline-none focus:ring-2 focus:ring-blue-500"
          />
        </div>

        {/* Address Section */}
        <div className="md:col-span-2 mt-4 bg-blue-100 px-4 py-2 rounded">
          <h3 className="text-lg font-semibold text-blue-800">Address</h3>
        </div>
        <div>
          <label className="block mb-1 font-medium text-gray-700">Street</label>
          <input
            type="text"
            name="street"
            value={formData.street}
            onChange={handleChange}
            required
            className="w-full px-4 py-2 border rounded-lg shadow-sm focus:outline-none focus:ring-2 focus:ring-blue-500"
          />
        </div>
        <div>
          <label className="block mb-1 font-medium text-gray-700">City</label>
          <input
            type="text"
            name="city"
            value={formData.city}
            onChange={handleChange}
            required
            className="w-full px-4 py-2 border rounded-lg shadow-sm focus:outline-none focus:ring-2 focus:ring-blue-500"
          />
        </div>
        <div>
          <label className="block mb-1 font-medium text-gray-700">State</label>
          <input
            type="text"
            name="state"
            value={formData.state}
            onChange={handleChange}
            required
            className="w-full px-4 py-2 border rounded-lg shadow-sm focus:outline-none focus:ring-2 focus:ring-blue-500"
          />
        </div>
        <div>
          <label className="block mb-1 font-medium text-gray-700">Postal Code</label>
          <input
            type="text"
            name="postal_code"
            value={formData.postal_code}
            onChange={handleChange}
            required
            className="w-full px-4 py-2 border rounded-lg shadow-sm focus:outline-none focus:ring-2 focus:ring-blue-500"
          />
        </div>
        <div>
          <label className="block mb-1 font-medium text-gray-700">Country</label>
          <input
            type="text"
            name="country"
            value={formData.country}
            onChange={handleChange}
            required
            className="w-full px-4 py-2 border rounded-lg shadow-sm focus:outline-none focus:ring-2 focus:ring-blue-500"
          />
        </div>

        {/* Identification Section */}
        <div className="md:col-span-2 mt-4 bg-indigo-100 px-4 py-2 rounded">
          <h3 className="text-lg font-semibold text-indigo-800">Identification</h3>
        </div>
        <div>
          <label className="block mb-1 font-medium text-gray-700">ID Type</label>
          <input
            type="text"
            name="id_type"
            value={formData.id_type}
            onChange={handleChange}
            required
            className="w-full px-4 py-2 border rounded-lg shadow-sm focus:outline-none focus:ring-2 focus:ring-indigo-500"
          />
        </div>
        <div>
          <label className="block mb-1 font-medium text-gray-700">ID Number</label>
          <input
            type="text"
            name="id_number"
            value={formData.id_number}
            onChange={handleChange}
            required
            className="w-full px-4 py-2 border rounded-lg shadow-sm focus:outline-none focus:ring-2 focus:ring-indigo-500"
          />
        </div>
        <div>
          <label className="block mb-1 font-medium text-gray-700">Expiry Date</label>
          <input
            type="date"
            name="expiry_date"
            value={formData.expiry_date}
            onChange={handleChange}
            required
            className="w-full px-4 py-2 border rounded-lg shadow-sm focus:outline-none focus:ring-2 focus:ring-indigo-500"
          />
        </div>

        {/* Submit Button */}
        <div className="md:col-span-2 text-center mt-6">
          <button
            type="submit"
            disabled={loading}
            className="bg-gradient-to-r from-blue-500 to-indigo-600 text-white px-8 py-3 rounded-full font-semibold hover:from-blue-600 hover:to-indigo-700 transition disabled:opacity-60"
          >
            {loading ? 'Saving...' : 'Save User'}
          </button>
        </div>
      </form>
    </div>
  );
}



In [ ]:
## server

// Required packages
const express = require('express');
const mysql = require('mysql2');
const bodyParser = require('body-parser');
const cors = require('cors');

const app = express();
const port = 3000;

app.use(bodyParser.json());
app.use(cors());

// MySQL connection
const db = mysql.createConnection({
  host: 'sql3.freesqldatabase.com',
  user: 'sql3774622',
  password: 'ugwzw1FQai',
  database: 'sql3774622'
});

db.connect(err => {
  if (err) throw err;
  console.log('Connected to MySQL database.');
});

// ---------- Data Access Object (DAO) Functions ---------- //

// Get all users with their address and identification
app.get('/users', (req, res) => {
  const sql = `
    SELECT
      u.id AS user_id, u.name, u.dob, u.gender, u.nationality, u.email, u.phone,
      a.street, a.city, a.state, a.postal_code, a.country,
      i.id_type, i.id_number, i.expiry_date
    FROM user u
    LEFT JOIN address a ON u.id = a.user_id
    LEFT JOIN identification i ON u.id = i.user_id
  `;

  db.query(sql, (err, results) => {
    if (err) return res.status(500).send(err);
    res.json(results);
  });
});

// Get a specific user by ID with address and identification
app.get('/users/:id', (req, res) => {
  const userId = req.params.id;
  const sql = `
    SELECT
      u.id AS user_id, u.name, u.dob, u.gender, u.nationality, u.email, u.phone,
      a.street, a.city, a.state, a.postal_code, a.country,
      i.id_type, i.id_number, i.expiry_date
    FROM user u
    LEFT JOIN address a ON u.id = a.user_id
    LEFT JOIN identification i ON u.id = i.user_id
    WHERE u.id = ?
  `;

  db.query(sql, [userId], (err, results) => {
    if (err) return res.status(500).send(err);
    res.json(results);
  });
});

// Insert new user with address and identification (with validation)
app.post('/users', (req, res) => {
  const { name, dob, gender, nationality, email, phone, address, identification } = req.body;

  // Validation
  const missingFields = [];
  const requiredUserFields = { name, dob, gender, email, phone };
  const requiredAddressFields = address || {};
  const requiredIDFields = identification || {};

  for (const [key, value] of Object.entries(requiredUserFields)) {
    if (!value || value.trim() === '') missingFields.push(`user.${key}`);
  }
  ['street', 'city', 'state', 'postal_code', 'country'].forEach(field => {
    if (!requiredAddressFields[field] || requiredAddressFields[field].trim() === '') {
      missingFields.push(`address.${field}`);
    }
  });
  ['id_type', 'id_number', 'expiry_date'].forEach(field => {
    if (!requiredIDFields[field] || requiredIDFields[field].trim() === '') {
      missingFields.push(`identification.${field}`);
    }
  });

  if (missingFields.length > 0) {
    return res.status(400).json({ error: `Missing required fields: ${missingFields.join(', ')}` });
  }

  const insertUserQuery = `
    INSERT INTO user (name, dob, gender, nationality, email, phone)
    VALUES (?, ?, ?, ?, ?, ?)
  `;

  db.query(insertUserQuery, [name, dob, gender, nationality, email, phone], (err, userResult) => {
    if (err) return res.status(500).json({ error: 'Failed to insert user', details: err });

    const userId = userResult.insertId;

    const insertAddressQuery = `
      INSERT INTO address (user_id, street, city, state, postal_code, country)
      VALUES (?, ?, ?, ?, ?, ?)
    `;

    const insertIdQuery = `
      INSERT INTO identification (user_id, id_type, id_number, expiry_date)
      VALUES (?, ?, ?, ?)
    `;

    db.query(insertAddressQuery, [
      userId,
      address.street,
      address.city,
      address.state,
      address.postal_code,
      address.country
    ], (err) => {
      if (err) return res.status(500).json({ error: 'Failed to insert address', details: err });

      db.query(insertIdQuery, [
        userId,
        identification.id_type,
        identification.id_number,
        identification.expiry_date
      ], (err) => {
        if (err) return res.status(500).json({ error: 'Failed to insert identification', details: err });

        res.status(201).json({ message: 'User, address, and identification added successfully', userId });
      });
    });
  });
});

// Update user
app.put('/users/:id', (req, res) => {
  const userId = req.params.id;
  const { name, dob, gender, nationality, email, phone } = req.body;
  const sql = 'UPDATE user SET name = ?, dob = ?, gender = ?, nationality = ?, email = ?, phone = ? WHERE id = ?';
  db.query(sql, [name, dob, gender, nationality, email, phone, userId], (err, result) => {
    if (err) return res.status(500).send(err);
    res.json({ message: 'User updated successfully' });
  });
});

// Delete user
app.delete('/users/:id', (req, res) => {
  const userId = req.params.id;
  const sql = 'DELETE FROM user WHERE id = ?';
  db.query(sql, [userId], (err, result) => {
    if (err) return res.status(500).send(err);
    res.json({ message: 'User deleted successfully' });
  });
});

app.get('/transactions/user/:id', (req, res) => {
  const userId = req.params.id;
  const sql = `
    SELECT * FROM transactions
    WHERE user_id = ?
    ORDER BY user_id DESC
  `;
  db.query(sql, [userId], (err, results) => {
    if (err) return res.status(500).json({ error: 'Failed to fetch transactions', details: err });
    res.json(results);
  });
});

// Start server
app.listen(port, () => {
  console.log(`Server is running on http://localhost:${port}`);
});